demo to inspect the content of reactome database 
the conda yaml contains redundant package.
install py2neo 

In [1]:
import os
from py2neo import  Graph
import pandas as pd
import numpy as np
import pprint
import itertools
pp = pprint.PrettyPrinter(indent=4)
uri = "bolt://localhost:7687"
graph = Graph(uri, auth=('neo4j', 'test'))

In [65]:
#get 'highly involved literatures' in reactome
query='''
MATCH (lr:LiteratureReference)<-[:literatureReference]-(rle:ReactionLikeEvent {speciesName: 'Homo sapiens'})
WITH lr, COUNT(rle) AS reactionCount, COLLECT(DISTINCT rle.dbId) AS reaction_db_ids
RETURN
 lr.displayName AS literature_display_name,
 lr.pubMedIdentifier AS pubMedIdentifier,
 reactionCount AS reaction_count,
 reaction_db_ids
ORDER BY reactionCount DESC
LIMIT 25
'''
files=graph.run(query).data()
files[0]['literature_display_name'],files[0]['pubMedIdentifier'],files[0]['reaction_count'],f"{files[0]['reaction_db_ids'][:10]}..."

('Mapping the proximity interaction network of the Rho-family GTPases reveals signalling pathways and regulatory mechanisms',
 31871319,
 46,
 '[9696266, 8981637, 9013009, 9013004, 8980691, 9696268, 9013110, 9013111, 9714361, 9018778]...')

In [4]:
# get the linked ReactionLikeEvent for the 'top relevant literature'
pmid=files[0]['pubMedIdentifier']
query='''
MATCH (lr:LiteratureReference)<-[:literatureReference]-(rle:ReactionLikeEvent)
WHERE lr.pubMedIdentifier={pmid} and rle.speciesName='Homo sapiens'
RETURN
rle.dbId AS id
'''
ids=[i['id'] for i in  graph.run(query.format(pmid=pmid)).data()]

In [26]:
#get the input - reaction -> out and its summation
query='''
    MATCH (x:DatabaseObject)<-[y:input|output|summation]-(rle:ReactionLikeEvent)
    WHERE rle.dbId={entity} and rle.speciesName='Homo sapiens'
    RETURN
    x AS entity,
    type(y) AS label,
    rle.displayName AS subject
    '''
hit=graph.run(query.format(entity=ids[0])).data()
print('('+' + '.join([i['entity']['displayName'] for i in hit if i['label']=='input'])+') --'+hit[0]['subject']+'--> (' + ' + '.join([i['entity']['displayName'] for i in hit if i['label']=='output']) +')')
print([i['entity']['text'] for i in hit if i['label']=='summation'][0])

(RND2 effectors [cytosol] + RND2:GTP [plasma membrane]) --RND2 binds effectors--> (RND2:GTP:RND2 effectors [plasma membrane])
Active GTP bound RND2 binds the following effectors:<br>ARHGAP5 (Wennerberg et al. 2003; Bagi et al. 2020)<br>FNBP1 (Fujita et al. 2002; Kakimoto et al. 2004; Bagci et al. 2020)<br>FRS2 (Harada et al. 2005)<br>FRS3 (Harada et al. 2005)<br>KCTD13 (Gladwyn Ng et al. 2016)<br>PLXND1 (Uesugi et al. 2009)<br>PRAG1 (Tanaka et al. 2006)<br>TNFAIP1 (Gladwyn Ng et al. 2015; Gladwyn Ng et al. 2016)<br>UBXN11 (Katoh et al. 2002)<br><br>RND2 binds to the following candidate effectors reported in the high throughput screen by Bagci et al. 2020 or reported in some but all studies:<br>ALDH3A2 (Bagci et al. 2020)<br>ANKRD26 (Bagci et al. 2020)<br>ARHGAP1 (Bagci et al. 2020)<br>ARHGAP35 (Wennerberg et al. 2003: binding to active RND2; Bagci et al. 2020: no binding to active RND2)<br>CAV1 (Bagci et al. 2020)<br>CKAP4 (Bagci et al. 2020)<br>DEPDC1B (Bagci et al. 2020)<br>DLG5 (Bag

In [60]:
# inspect the abstract_input
abstract_input_id=hit[2]['entity']['dbId']
query=f'''
    MATCH (x:DatabaseObject)-[y]->(z:DatabaseObject)
    WHERE x.dbId={abstract_input_id}
    RETURN type(y) as typey
    '''
_=graph.run(query).data()
set([i['typey'] for i in _])

{'compartment', 'hasCandidate', 'hasMember', 'inferredTo', 'species'}

In [62]:
# check the member
query='''
    MATCH (x:DatabaseObject)-[y:hasMember]->(z:DatabaseObject)
    WHERE x.dbId=9696290
    RETURN z
    '''
for i in graph.run(query):
    print(i['z']['displayName'])

TNFAIP1 [cytosol]
FRS2 [plasma membrane]
UBXN11 [cytosol]
FNBP1 [cytosol]
KCTD13 [cytosol]
PLXND1 [plasma membrane]
PRAG1 [cytosol]
FRS3 [plasma membrane]
ARHGAP5 [cytosol]
